In [3]:
import json
import os
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import time
import urllib3
urllib3.disable_warnings()
import tqdm.notebook as tqdm 

class SnapshotScapper:
    def __init__(self, base_url, headers=None, outpath='.'):
        self.base_url = base_url
        if not headers:
            self.headers = {
                "Content-type": "application/json",
            }
        else:
            self.headers = headers
            
        self.sample_transport = RequestsHTTPTransport(
            url=self.base_url,
            use_json=True,
            headers=self.headers,
            verify=False,
            retries=3,
        )

        self.client = Client(
            transport=self.sample_transport,
            fetch_schema_from_transport=True,
        )
        
        self.outpath = outpath
        self.results_path = os.path.join(outpath, 'results')
        self.spaces_path = os.path.join(self.results_path, 'spaces')
        
        os.makedirs(self.spaces_path, exist_ok=True)
        
    def parse(self):
        spaces = self.get_spaces(data=[])
        
        with open(os.path.join(self.results_path, 'spaces.json'), 'w')  as f:
            f.write(json.dumps(spaces))
            
        for space in tqdm.tqdm(spaces):
            self.parse_space(space['id'])
    
    def parse_space(self, space_id):
        result = {
            'id': space_id,
            'proposals': []
        }
        space_path = os.path.join(self.spaces_path, space_id)
        try:
            os.mkdir(os.path.join(self.spaces_path, space_id))
        except:
            pass
        proposals = self.get_proposals([space_id], data=[])
        for proposal in tqdm.tqdm(proposals):
            if proposal['votes'] > 0:
                votes = self.get_votes(proposal['id'], data=[])
                if not votes:
                    print(proposal)
            else:
                votes = []
                
            proposal['votes_data'] = votes
            with open(os.path.join(space_path, proposal['id']+'.json'), 'w') as f:
                f.write(json.dumps(proposal))
            result['proposals'].append(proposal)
        return result

    def get_votes(self, proposal_id, data=[], skip=0, retry=0):
        if retry > 10:
            return data
        try:
            # The string formating of strings woth curly braces in python is uggly {{ and }} instead of the simple one
            # I took the decision to use the old school % formating because its just one number and makes reading the query easier !
            # also, 100 is the max return number
            # spaces is a list like ["balancer", "yam.eth"]
            query = '''
                query Votes {
                  votes (
                    first: 1000
                    skip:%s
                    where: {
                      proposal: "%s"
                    }
                  ) {
                    id
                    voter
                    created
                    choice
                    space {
                      id
                    }
                  }
                }
            ''' % (skip, proposal_id)
            
            query = gql(query.replace("'", '"')) 

            result = self.client.execute(query)
            data += result['votes']
            if len(result['votes']) == 1000:
                time.sleep(0.5)
                return self.get_votes(proposal_id, data=data, skip=skip+1000)
            return data
        except:
            time.sleep(retry+1)
            return self.get_votes(proposal_id, data=data, skip=skip, retry=retry+1)    
    
    def get_proposals(self, spaces, data=[], skip=0, retry=0):
        if retry > 10:
            return data
        try:
            # The string formating of strings woth curly braces in python is uggly {{ and }} instead of the simple one
            # I took the decision to use the old school % formating because its just one number and makes reading the query easier !
            # also, 100 is the max return number
            # spaces is a list like ["balancer", "yam.eth"]
            query = '''
                query Proposals {
                  proposals(
                    first: 100,
                    skip: %s,
                    where: {
                      space_in: %s,   
                      state: "closed"
                    },
                    orderBy: "created",
                    orderDirection: desc
                  ) {
                    id
                    title
                    body
                    choices
                    start
                    end
                    snapshot
                    state
                    author
                    space {
                      id
                      name
                    }
                    votes
                  }
                }
            ''' % (skip, spaces)
            
            query = gql(query.replace("'", '"')) 

            result = self.client.execute(query)
            data += result['proposals']
            if len(result['proposals']) == 100:
                time.sleep(0.5)
                return self.get_proposals(spaces, data=data, skip=skip+100)
            return data
        except:
            time.sleep(retry+1)
            return self.get_proposals(spaces, data=data, skip=skip, retry=retry+1)
            
    def get_spaces(self, data=[], skip=0, retry=0):
        if retry > 10:
            return data
        try:
            # The string formating of strings woth curly braces in python is uggly {{ and }} instead of the simple one
            # I took the decision to use the old school % formating because its just one number and makes reading the query easier !
            # also, 100 is the max return number
            query = gql('''
                query Spaces {
                  spaces(
                    first: 100,
                    skip: %s,
                    orderBy: "created",
                    orderDirection: desc
                  ) {
                    id
                    name
                    about
                    network
                    symbol
                    strategies {
                      name
                      params
                    }
                    admins
                    members
                    filters {
                      minScore
                      onlyMembers
                    }
                    plugins
                  }
                }
            ''' % skip) 

            result = self.client.execute(query)
            data += result['spaces']
            if len(result['spaces']) == 100:
                time.sleep(0.5)
                return self.get_spaces(data=data, skip=skip+100)
            return data
        except:
            time.sleep(retry+1)
            return self.get_spaces(data=data, skip=skip, retry=retry+1)
            

In [4]:
scrapper = SnapshotScapper('https://hub.snapshot.org/graphql')

In [31]:
data = scrapper.parse_space('yam.eth')

  0%|          | 0/149 [00:00<?, ?it/s]

In [33]:
scrapper.parse()

  0%|          | 0/3488 [00:00<?, ?it/s]

TypeError: parse_space() got an unexpected keyword argument 'data'

In [10]:
len(data['proposals'][0]['votes_data'])

46

In [17]:
votes = scrapper.get_votes('QmRLiSZdXJLNaejrgpAL5bqzYefMxc4JJJ1GZSg9GtiCSW', data=[])

In [14]:
for p in data['proposals']:
    if not p['votes_data']:
        print(p)
    if p['votes'] != len(p['votes_data']):
        print(p)

{'id': 'QmRLiSZdXJLNaejrgpAL5bqzYefMxc4JJJ1GZSg9GtiCSW', 'title': 'YIP 77: Bootstrapping Yam Fuse Pools', 'body': '**Yam Dao has been whitelisted as a ‘Fuse Pool Creator’:**\nhttps://snapshot.org/#/rari/proposal/QmdNYSDUdojyAREMf2hzf2nEw7iRTSbvrSsVr4hWX2LHG5\n\nRari Capital’s Fuse product, allows for the creation of custom lending/borrowing markets which are parameterised by the pool creators, in our case this would be Yam Dao. By creating a Fuse pool centered around the YAM token the community can reap the following benefits:\n\n**Increase token utility:**\n\nBorrow stablecoins against YAM collateral\nBorrow YAM or YAM/ETH SLP against stables to farm without price risk\nBorrow YAM or YAM/ETH SLP against stables to hedge YAM positions\n\n**Make YAM a yield bearing asset:**\n\nExisting YAM or newly minted YAM would be yield bearing if deposited into a Fuse pool maintained by Yam.\n\n**Earn yield on a portion of treasury assets:**\n\nAny Treasury assets deposited into the pool would also

TypeError: object of type 'NoneType' has no len()

In [6]:
for i in tqdm.tqdm(range(10000000)):
    pass

  0%|          | 0/10000000 [00:00<?, ?it/s]

In [79]:
proposals = scrapper.get_proposals(["yam.eth"])

/home/xqua/anaconda3/envs/datascience/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'hub.snapshot.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/xqua/anaconda3/envs/datascience/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'hub.snapshot.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [100]:
proposals[1]

{'id': '0xdaf98ef5a14a8aeac30f3adf7305061aa25d912556490ab32ff6e18059ed8c27',
 'title': 'YIP-91A Polygon Liquidity ',
 'body': 'Forum Post: https://forum.yam.finance/t/yip-91-yam-synths-polygon-liquidity-strategy/1551\n\n### Basic Summary\nWe need to provide liquidity to our New LSP synths on Polygon when they are ready (expected early November 2021). In order to do this we must set funds from the treasury, which lives on Ethereum mainnet to polygon, and then manage those funds there\n\n### Abstract - What am I proposing?\n#### Liquidity allocation\nIn continuation of the asset allocation from [YIP-83](https://forum.yam.finance/t/yip-83-re-allocation-of-assets-post-rebalancing/1518), I propose that we send 130 ETH and $1.1M in stablecoins to provide liquidity for our new synths on Polygon.\n\n### Motivation - Why am I proposing it?\nWe are launching new Synths on polygon as part of a rollout of our new DeFi-Tools product line. We need to continue bootstrapping these pools to grow the Ya

In [101]:
spaces = scrapper.get_spaces()

/home/xqua/anaconda3/envs/datascience/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'hub.snapshot.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/xqua/anaconda3/envs/datascience/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'hub.snapshot.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/xqua/anaconda3/envs/datascience/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'hub.snapshot.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  w

In [63]:
len(spaces)

3485

In [64]:
filtered = [s for s in spaces if len(s['members']) > 0]

In [66]:
filtered

[{'id': 'zfbapp.eth',
  'name': 'YANGTUOTUO',
  'about': '',
  'network': '1',
  'symbol': 'YTB',
  'strategies': [{'name': 'erc20-balance-of',
    'params': {'symbol': 'YTB',
     'address': '0xD4203e6fFC798765Dea121D795b2a56684c1c738',
     'decimals': 18}},
   {'name': 'erc721',
    'params': {'symbol': 'YTB',
     'address': '0xD4203e6fFC798765Dea121D795b2a56684c1c738'}}],
  'admins': ['0xD4203e6fFC798765Dea121D795b2a56684c1c738'],
  'members': ['0xD4203e6fFC798765Dea121D795b2a56684c1c738'],
  'filters': {'minScore': 5, 'onlyMembers': False},
  'plugins': {}},
 {'id': 'ethfund.eth',
  'name': 'ETHFUND',
  'about': '',
  'network': '1',
  'symbol': 'eth',
  'strategies': [{'name': 'erc20-balance-of',
    'params': {'symbol': 'eth',
     'address': '0x6b175474e89094c44da98b954eedeac495271d0f',
     'decimals': 18}}],
  'admins': ['0x26427279D847EcCe6f09bF3E629A06dfD433898E'],
  'members': ['0x26427279D847EcCe6f09bF3E629A06dfD433898E'],
  'filters': {'minScore': 1e-06, 'onlyMembers': 

In [65]:
len(filtered)

1840